#Criando um classificador de textos

###Para isso vamos importar algumas bibliotecas.

#####*CountVectorizer*: para transformar um texto em vetor.
#####*TfidfTransformer*: para fazer a normalização dos dados.
#####*MultinomialNB*: para assumir a independência total das features do modelo.
#####*Pipeline*: para trabalhar com uma seguência de tarefas
#####*SnowballStemmer*: para reduzir uma palavra para a sua forma base.
#####*SGDClassifier*: para as previsões finais do pipeline.
#####*GridSearchCV*: para automatizar o ajuste dos processos.
#####*f1_score*: para a visualização a precisão e recall juntas.
#####*accuracy_score*: para caulo (todos os acertos / total).
#####*confusion_matrix*: tabela para auxiliar a avaliação do modelo.
#####*classification_report*: retornar um relatório.
#####*ConfusionMatrixDisplay*: exibe em forma de imagem.

In [1]:
from sklearn.datasets import fetch_20newsgroups 
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer 
from sklearn.naive_bayes import MultinomialNB 
from sklearn.pipeline import Pipeline 
from nltk.stem.snowball import SnowballStemmer 
import numpy as np
from sklearn.linear_model import SGDClassifier 
from sklearn.model_selection import GridSearchCV
import nltk 
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
import warnings
import matplotlib.pyplot as plt 
warnings.simplefilter('ignore')

#Carregando o Dataset

#####Vamos usar o dataset fetch 20 new groups pois ele retorna uma lista de textos divididos em 20 grupos para serem alimentados por extratores de textos.

In [179]:
newsgroups = fetch_20newsgroups(subset='train')

####Mostrando os grupos dentro do dataset.

In [180]:
list(newsgroups.target_names)

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

####Escolhendo 10 grupos e fazendo o treino e teste.

In [181]:
categories = ['alt.atheism', 'soc.religion.christian','rec.motorcycles','rec.autos','comp.graphics','comp.os.ms-windows.misc','comp.sys.ibm.pc.hardware','sci.electronics','talk.politics.guns','sci.space',] 
df_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)
df_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True)

##Apresentando as classes (grupos)

In [182]:
df_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'rec.autos',
 'rec.motorcycles',
 'sci.electronics',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns']

###convertendo para matriz uma coleção de texto, produzindo uma representação das contagens instanciando o algoritimo do count vectorizer. 

In [183]:
count_vect = CountVectorizer() 
X_train_counts = count_vect.fit_transform(df_train.data)
X_train_counts.shape

(5766, 92584)

In [184]:
tfidf_transformer = TfidfTransformer() 
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

#Abordagem 1
####*Treinando um modelo*

#####Utilizando o pipeline para as tarefaz serem execultadas em seguência.

In [185]:
clf = MultinomialNB() 
clf.fit(X_train_tfidf, df_train.target)

MultinomialNB()

#Pipeline

#####Passo 1, aplicar o count vectorizer nos textos 
#####Passo 2, aplicar o TFIDF nos textos
#####Passo 3, aplicar o algoritmo Naive Bayes

In [186]:
clf_1 = Pipeline([
    ('vect', CountVectorizer()),    
    ('tfidf', TfidfTransformer()), 
    ('clf', MultinomialNB())])

###Treinando o modelo no pipeline.

In [187]:
clf_trained = clf_1.fit(df_train.data, df_train.target)

###Fazendo predição no dados de teste.

In [188]:
pred = clf_trained.predict(df_test.data)

In [189]:
acc = np.mean(pred == df_test.target)
print('>>>> Acurácia: ', acc)

>>>> Acurácia:  0.8232994526974199


###Gerando as métricas de acerto do modelo.

In [190]:
creport = classification_report(df_test.target, pred, target_names=df_test.target_names)
print(creport)

                          precision    recall  f1-score   support

             alt.atheism       0.98      0.54      0.69       319
           comp.graphics       0.89      0.74      0.81       389
 comp.os.ms-windows.misc       0.88      0.68      0.77       394
comp.sys.ibm.pc.hardware       0.75      0.84      0.79       392
               rec.autos       0.89      0.95      0.92       396
         rec.motorcycles       0.93      0.94      0.93       398
         sci.electronics       0.89      0.66      0.76       393
               sci.space       0.88      0.91      0.89       394
  soc.religion.christian       0.57      0.98      0.73       398
      talk.politics.guns       0.88      0.95      0.91       364

                accuracy                           0.82      3837
               macro avg       0.85      0.82      0.82      3837
            weighted avg       0.85      0.82      0.82      3837



####Nesse modelo utilizando 10 classes (grupos) conseguimos um recall baixo para a metade, vamos tentar da uma melhorada com o tuning.

##Abordagem 2

###***Tuning***

####Tuning é uma forma de otimizar os dados para retornar valores mais autos.

####Criando uma lista de parâmetros para ajuste.

In [191]:
parameters = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False), 'clf__alpha': (1e-2, 1e-3)}

####Buscando os melhores parâmetros e treinando o modelo.

In [192]:
gs_clf = GridSearchCV(clf_trained, parameters, n_jobs=-1)  
gs_clf = gs_clf.fit(df_train.data, df_train.target)  

####Mostrando a melhor pontuação.

In [193]:
print(gs_clf.best_score_)
gs_clf.best_params_

0.9323625655925843


{'clf__alpha': 0.001, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 2)}

####depois do tuning temos um valor aceitavel.

==================================================

####Prediçao no dado de teste.



In [194]:
pred = gs_clf.predict(df_test.data)

In [195]:
acc = np.mean(pred == df_test.target)
print('>>>> Acurácia: ', acc)

>>>> Acurácia:  0.8665624185561637


###🤔A ideia era ele apresentar um valor mais aulto, para isso é so mudar o modelo.

In [196]:
creport = classification_report(df_test.target, pred, target_names=df_test.target_names)
print(creport)

                          precision    recall  f1-score   support

             alt.atheism       0.97      0.88      0.92       319
           comp.graphics       0.80      0.82      0.81       389
 comp.os.ms-windows.misc       0.82      0.65      0.72       394
comp.sys.ibm.pc.hardware       0.69      0.82      0.75       392
               rec.autos       0.92      0.92      0.92       396
         rec.motorcycles       0.96      0.95      0.95       398
         sci.electronics       0.84      0.77      0.80       393
               sci.space       0.91      0.93      0.92       394
  soc.religion.christian       0.88      0.96      0.92       398
      talk.politics.guns       0.94      0.98      0.96       364

                accuracy                           0.87      3837
               macro avg       0.87      0.87      0.87      3837
            weighted avg       0.87      0.87      0.87      3837



##Abordagem 3

######Vamos treinar um segundo modelo 🤞.

###***Treinando um segundo modelo***

In [197]:
clf_2 = Pipeline([
    ('vect', CountVectorizer()), 
    ('tfidf', TfidfTransformer()), 
    ('clf-svm', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, max_iter=25, random_state=42))]) 

In [198]:
svm_trained = clf_2.fit(df_train.data, df_train.target) 

In [199]:
pred_1 = svm_trained.predict(df_test.data)

In [200]:
acc_2 = np.mean(pred_1 == df_test.target)
print('>>>> Acurácia: ', acc_2)

>>>> Acurácia:  0.8759447485014334


In [201]:
creport_2 = classification_report(df_test.target, pred_1, target_names=df_test.target_names)
print(creport_2)

                          precision    recall  f1-score   support

             alt.atheism       0.91      0.77      0.84       319
           comp.graphics       0.90      0.84      0.87       389
 comp.os.ms-windows.misc       0.82      0.84      0.83       394
comp.sys.ibm.pc.hardware       0.78      0.79      0.78       392
               rec.autos       0.91      0.94      0.93       396
         rec.motorcycles       0.93      0.97      0.95       398
         sci.electronics       0.89      0.72      0.79       393
               sci.space       0.89      0.96      0.92       394
  soc.religion.christian       0.82      0.96      0.89       398
      talk.politics.guns       0.94      0.96      0.95       364

                accuracy                           0.88      3837
               macro avg       0.88      0.87      0.87      3837
            weighted avg       0.88      0.88      0.87      3837



##Abordagem 4

###***Tuning***


In [204]:
parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False),'clf-svm__alpha': (1e-2, 1e-3)}

In [205]:
gs_clf_svm = GridSearchCV(svm_trained, parameters_svm, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(df_train.data, df_train.target)

In [206]:
print(gs_clf_svm.best_score_)
gs_clf_svm.best_params_

0.9247315044319617


{'clf-svm__alpha': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}

In [207]:
pred = gs_clf_svm.predict(df_test.data)

In [208]:
acc = np.mean(pred == df_test.target)
print('>>>> Acurácia: ', acc)

>>>> Acurácia:  0.870471722700026


In [209]:
creport = classification_report(df_test.target, pred, target_names=df_test.target_names)
print(creport)

                          precision    recall  f1-score   support

             alt.atheism       0.89      0.78      0.83       319
           comp.graphics       0.89      0.81      0.85       389
 comp.os.ms-windows.misc       0.83      0.82      0.82       394
comp.sys.ibm.pc.hardware       0.78      0.79      0.79       392
               rec.autos       0.92      0.92      0.92       396
         rec.motorcycles       0.92      0.97      0.95       398
         sci.electronics       0.91      0.71      0.80       393
               sci.space       0.87      0.95      0.91       394
  soc.religion.christian       0.80      0.96      0.87       398
      talk.politics.guns       0.91      0.97      0.94       364

                accuracy                           0.87      3837
               macro avg       0.87      0.87      0.87      3837
            weighted avg       0.87      0.87      0.87      3837

